## Testing basic LSTM

In [20]:
# create sample data
# try to learn fruit names letter-wise

words = ['apple', 'banana', 'pear', 'lime', 'cherry', 'orange', 'berry', 'strawberry']

# sequence stop sign
SEQUENCE_STOP_SIGN = '.'

# transform words to letter arrays
words_arrays = [[l for l in w] for w in words]

In [26]:
# create sequences in many-to-many configuration
sequences_x, sequences_y = zip(*[(w, w[1:] + [SEQUENCE_STOP_SIGN]) for w in words_arrays])
sequences_x[0], sequences_y[0:1]

((['a', 'p', 'p', 'l', 'e'],), (['p', 'p', 'l', 'e', '.'],))

In [32]:
# one-hot encode letters (and stop sign)
import string
ALPHABET = string.ascii_lowercase + SEQUENCE_STOP_SIGN
ALPHABET_LENGTH = len(ALPHABET)

import numpy as np
def encode_letter(letter):
    res = np.zeros(ALPHABET_LENGTH)
    res[ALPHABET.index(letter)] = 1
    return res

def decode_letter(ohe_vec):
    # for now, one dimention only
    return ALPHABET[ohe_vec.argmax()]

encode_letter('d')

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
def encode_word(word):
    return np.stack([encode_letter(l) for l in word])
    
def decode_word(ohe_word):
    return ''.join([decode_letter(ohe_letter) for ohe_letter in ohe_word])

encode_word('asd')

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
# encode dataset
data_x = [encode_word(seq) for seq in sequences_x]